# Parallelization con LangGraph

## Definición (nivel MIT/Stanford, aplicada)
Parallelization divide en subproblemas independientes y ejecuta ramas al mismo tiempo para reducir tiempo de respuesta.

## Cuándo usarla
Úsala cuando opener, follow-up y guardrails pueden calcularse en paralelo sin dependencia fuerte.

## Cuándo NO usarla
Evítala cuando las ramas dependen del mismo razonamiento acumulado; ahí conviene secuencial.

## Diagrama Conceptual

```mermaid
graph TD
A[Contexto] --> B[Rama Opener]
A --> C[Rama Follow-up]
A --> D[Rama Tono]
B --> E[Agregador]
C --> E
D --> E
```

In [ ]:
# Setup de notebook
from __future__ import annotations

import importlib.util
import inspect
import json
import sys
from pathlib import Path
from IPython.display import Markdown, display

# Add project root to path
project_root = Path.cwd()
while not (project_root / "pyproject.toml").exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

def load_module(path: Path, module_name: str):
    spec = importlib.util.spec_from_file_location(module_name, path)
    if spec is None or spec.loader is None:
        raise RuntimeError(f"Cannot load module: {path}")
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

ROOT = project_root
print('Repo root:', ROOT)

In [ ]:
script_path = ROOT / '04_langchain_langgraph/02_parallelization/Notebooks/01_parallelization_langgraph.py'
module = load_module(script_path, 'nb_parallel')
run_fn = module.run_parallelization
print('Script:', script_path)
print('Función:', run_fn.__name__)

## Código de la arquitectura (visible en notebook)

In [ ]:
print(inspect.getsource(run_fn)[:5000])

## Ejemplo 1: caso base del curso

In [ ]:
result_1 = run_fn(verbose=False)
print('Modelo:', result_1['__model'])
print('Context hash:', result_1['__context_hash'])

In [ ]:
print('Salida final ejemplo 1:')
print(json.dumps(result_1['final'], ensure_ascii=False, indent=2))

## Grafo real LangGraph (comportamiento)

In [ ]:
agent = result_1['__agent']
try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as exc:
    print('No se pudo renderizar PNG en este entorno:', exc)
    display(Markdown('```mermaid\n' + result_1['__graph_mermaid'] + '\n```'))

## Ejemplo 2: asistente latino experto en enamorar (coqueteo creativo y respetuoso)

In [ ]:
profile_coqueteo = {
  "tipo_persona": "asistente latino experto en enamorar con elegancia",
  "gustos": [
    "salsa romantica",
    "cafes bohemios",
    "poesia urbana",
    "paseos nocturnos"
  ],
  "estilo": "carismatico, coqueto, respetuoso, humor picante sutil",
  "contexto": "quiere iniciar una conversacion de coqueteo sin caer en frases prefabricadas"
}
result_2 = run_fn(profile=profile_coqueteo, verbose=False)
print('Context hash coqueteo:', result_2['__context_hash'])

In [ ]:
print('Salida final ejemplo coqueteo:')
print(json.dumps(result_2['final'], ensure_ascii=False, indent=2))

## Errores típicos en producción
- Elegir esta arquitectura por moda y no por necesidad.
- No definir métricas de calidad/costo/latencia antes de escalar.
- No versionar contexto ni criterios de evaluación.
- Falta de observabilidad para diagnosticar degradaciones.

## Lectura crítica del resultado
- Evalúa si el flujo realmente mejoró calidad o solo añadió complejidad.
- Compara latencia/costo vs beneficio de control arquitectónico.
- Decide si esta arquitectura es mínima suficiente para tu caso real.